## Informe - Proyecto 3: Tokenización y Embeddings Visuales

### Objetivo
Analizar cómo diferentes técnicas de tokenización influyen en las representaciones semánticas generadas por modelos de lenguaje basados en Transformers


### Metodología

Se trabajó con un corpus de 20 frases en español relacionadas al procesamiento de lenguaje natural. Se compararon dos métodos de tokenización:

1. **Tokenización por espacios**: Separación clásica con limpieza básica (minúsculas, eliminación de signos y tildes).
2. **Byte-Pair Encoding (BPE)**: Entrenado con SentencePiece sobre el corpus.

Luego, se aplicaron los siguientes pasos:

- Generación de embeddings usando el modelo `dccuchile/bert-base-spanish-wwm-uncased`
- Cálculo de la matriz de similitud coseno para cada tokenización
- Proyección 2D mediante PCA para visualizar agrupaciones

In [3]:
import pandas as pd
import numpy as np

# Cargar matrices de similitud 
sim_bpe = pd.read_csv("resultados/similitud_bpe.csv", index_col=0)
sim_esp = pd.read_csv("resultados/similitud_espacios.csv", index_col=0)

# Convertir a matrices numpy
mat_bpe = sim_bpe.values
mat_esp = sim_esp.values

# Rellenar diagonal con -1 para ignorar autocomparación
np.fill_diagonal(mat_bpe, -1)
np.fill_diagonal(mat_esp, -1)

# Función para extraer los top-k pares más similares
def obtener_pares_mas_similares(matriz, k=5):
    pares = []
    for _ in range(k):
        i, j = np.unravel_index(np.argmax(matriz), matriz.shape)
        valor = matriz[i, j]
        pares.append((f"Frase {i+1} - Frase {j+1} (sim={valor:.3f})"))
        matriz[i, j] = -1  # Evitar repetir
        matriz[j, i] = -1
    return pares

# Extraer resultados
pares_bpe = obtener_pares_mas_similares(mat_bpe.copy())
pares_esp = obtener_pares_mas_similares(mat_esp.copy())

tabla_similares = pd.DataFrame({
    "Tokenización BPE": pares_bpe,
    "Tokenización por Espacios": pares_esp
})

import IPython.display as display
display.display(tabla_similares)


,Tokenización BPE,Tokenización por Espacios
0,Frase 4 - Frase 13 (sim=0.966),Frase 17 - Frase 19 (sim=0.846)
1,Frase 4 - Frase 17 (sim=0.956),Frase 5 - Frase 13 (sim=0.805)
2,Frase 16 - Frase 20 (sim=0.956),Frase 13 - Frase 17 (sim=0.722)
3,Frase 17 - Frase 19 (sim=0.956),Frase 16 - Frase 17 (sim=0.706)
4,Frase 4 - Frase 11 (sim=0.955),Frase 8 - Frase 18 (sim=0.704)


### Conclusiones

Los resultados evidencian que la técnica de tokenización influye directamente en las representaciones generadas por modelos Transformer

- La tokenización BPE permitió al modelo capturar similitudes semánticas de manera más precisa, generando valores más altos en la matriz de similitud coseno entre frases relacionadas

- La tokenización por separación de espacios mostró una menor capacidad para agrupar frases similares, con valores de similitud generalmente más bajos

Estas diferencias también se reflejan visualmente en los gráficos de PCA, ya que las frases tokenizadas con BPE tienden a formar clústeres más compactos y coherentes, mientras que las generadas por espacios están más dispersas
